In [ ]:
library(tidyverse)
library(xtable)
library(extrafont)
#font_import() #only run this once when exporting. Make sure youe h
loadfonts()
fonts()

In [ ]:
library(RColorBrewer)
custom_pal <- brewer.pal(5, "Set1")[c(2,3,4)]

custom_theme <- theme_bw() +
        theme(panel.border = element_blank(),
        panel.grid.major.x = element_blank(),
        panel.grid.minor.x = element_blank())

### Data Loading

In [ ]:
source("data_load.R")

In [ ]:
df <- load_from_path("../data") %>% 
    mutate(multi = factor(multi)) %>%
    filter(max_peak %in% c(38,40,42,45,50))

### Overview 

Some overviews on the result data set

In [ ]:
df[301:310,]

Summary for a specific scenario

In [ ]:
df %>% 
    filter(multi==1, aggregation_interval == 5, max_peak == 40, aggregation_type == "average") %>% 
    select(day, C, C_reference, R, R_rel, fft1, fft8n) %>%
    summary()

Number of days where optimization problem with a given `max_peak` is infeasible, and where no battery is needed

In [ ]:
df %>%
    filter(aggregation_interval == 5, aggregation_type == "average") %>%
    group_by(max_peak, aggregation_interval) %>%    
    mutate(C_state = case_when(is.na(C) ~ "Infeasible",
                         C == 0 ~ "No Battery Needed",
                         C > 0 ~ "Battery Size > 0")) %>%
    group_by(max_peak, aggregation_interval, C_state) %>%    
    summarize(n = n()) %>%
    spread(C_state, n) %>%
    #mutate(Infeasible = replace_na(Infeasible, 0)) %>%
    arrange(max_peak, aggregation_interval)

In [ ]:
df %>%
    filter(aggregation_interval >= 60, aggregation_interval <= 900, aggregation_type == "average") %>%
    group_by(max_peak, aggregation_interval) %>%    
    mutate(C_state = case_when(is.na(C) ~ "Infeasible",
                         C == 0 ~ "No Battery Needed",
                         C > 0 ~ "Battery Size > 0")) %>%
    group_by(max_peak, aggregation_interval, C_state) %>%    
    summarize(n = n()) %>%
    spread(C_state, n) %>%
    mutate(Infeasible = replace_na(Infeasible, 0)) %>%
    arrange(max_peak, aggregation_interval)

In relative frequencies

In [ ]:
df %>%
    filter(aggregation_interval >= 60, aggregation_interval <= 900, aggregation_type == "average") %>%
    group_by(max_peak, aggregation_interval) %>%    
    mutate(C_state = case_when(is.na(C) ~ "Infeasible",
                         C == 0 ~ "No Battery Needed",
                         C > 0 ~ "Battery Size > 0")) %>%
    group_by(max_peak, aggregation_interval, C_state) %>%    
    summarize(n = n()) %>%
    mutate(freq = round(n / sum(n), 2)) %>%
    select(-n)%>%
    spread(C_state, freq) %>%
    mutate(Infeasible = replace_na(Infeasible, 0)) %>%
    arrange(max_peak, aggregation_interval)

### Results

#### Filtering

Filter for feasible solutions with positive battery size

In [ ]:
df_res <- df %>% 
    filter(!is.na(C), C > 0)

Actual battery capacities in the data

In [ ]:
df_res %>%
    filter(aggregation_interval == 5, max_peak == 40) %>%    
    summary()
#    mutate(freq = round(n / sum(n), 2)) %>%
#    select(-n)%>%
#   spread(C_state, freq) %>%
#    mutate(Infeasible = replace_na(Infeasible, 0)) %>%
#    arrange(max_peak, aggregation_interval)

#### Comparison of R on different sampling rates

The scaling of the factory affects the battery dimension by the same factor.

In [ ]:
df_res %>% filter(aggregation_type == "average", aggregation_interval == 900) %>% 
    group_by(multi, aggregation_interval, max_peak) %>% 
    summarize(max_C = max(C))%>%
    mutate(scaled_max_C = max_C / as.numeric(levels(multi)[multi])) %>%
    ungroup() %>%
    select(multi, max_peak, scaled_max_C) %>% 
    spread(multi, scaled_max_C, sep = "_")

In [ ]:
df_res <- df_res %>% filter(multi == 1)

Comparison of different `max_peak` and `aggregation_interval`

In [ ]:
options(repr.plot.width=6, repr.plot.height=5)
df_res %>%
    filter(aggregation_interval > 0) %>%
    ggplot() +
    geom_boxplot(aes(x=factor(aggregation_interval), y=R, color=factor(aggregation_type))) +
    facet_grid(rows = vars(max_peak), labeller = purrr::partial(label_both, sep = " = ")) +
    labs(x="Sampling Period [s]", y="R [kWh]", color="Sampling Mode") +
    scale_colour_manual(values=custom_pal) +
    custom_theme +
    theme(strip.background = element_rect(colour = FALSE, fill = "white"),
          panel.spacing = unit(2, "lines"))

#### Relative Error

In [ ]:
options(repr.plot.width=6, repr.plot.height=5)
df_res %>%
    filter(aggregation_interval > 0) %>%
    ggplot() +
    geom_boxplot(aes(x=factor(aggregation_interval), y=R_rel, color=factor(aggregation_type))) +
    facet_grid(rows = vars(max_peak), labeller = purrr::partial(label_both, sep = " = ")) +
    labs(x="Sampling Period [s]", y="relative Error", color="Sampling Mode") +
    scale_colour_manual(values=custom_pal) +
    custom_theme +
    theme(strip.background = element_rect(colour = FALSE, fill = "white"),
          panel.spacing = unit(2, "lines"))

# Detailed plot for `max_peak = 40`

In [ ]:
options(repr.plot.width=8, repr.plot.height=3)
df_res %>%
    filter(aggregation_interval > 0, aggregation_interval != 2700, max_peak == 40) %>%
    ggplot(aes(x=factor(aggregation_interval), y=R, color=factor(aggregation_type))) +
    geom_boxplot(outlier.shape = NA, lwd=.4) +
    geom_point(position = position_jitterdodge(), alpha=0.3, size=.25) +
    scale_colour_manual(values=custom_pal) +
    labs(x="Sampling Period [s]", y="R [kWh]", color="Sampling Mode") +
    guides(col = guide_legend(nrow = 1, byrow = TRUE)) +
    custom_theme +
    theme(axis.title=element_text(size=8),
          axis.title.x=element_text(margin = margin(t = 5, r = 0, b = 0, l = 0)),
          axis.title.y=element_text(margin = margin(t = 0, r = 0, b = 0, l = 0)),
          legend.title=element_text(size=8),
          legend.text=element_text(size=8),
          legend.position=c(0.30, 0.80),
          legend.margin=margin(c(0,0,0,0)),
          axis.text=element_text(size=8),
         )

In [ ]:
ggsave("export/plot_sampling_periods.pdf", 
       plot = last_plot(), 
       width = 85, 
       height = 45,
       units = "mm",
       family = "Linux Libertine Display")

# Extract for `aggregation_interval = 900`

In [ ]:
options(repr.plot.width=2, repr.plot.height=3)
df_res %>%
    filter(aggregation_type == "average", aggregation_interval == 900) %>%
    ggplot(aes(x=factor(max_peak), y=R, fill='constant', color='constant')) +
    ylim(-3.5, 3.5) +
    guides(fill=FALSE, color=FALSE) +
    geom_boxplot(outlier.shape = NA, lwd=.4) +
    geom_point(alpha=0.4, position = position_jitterdodge(), size=.5) +
    labs(x="target maximum peak [kW]", y="R [kWh]", color="Max Peak", caption="average") +
    custom_theme +
    scale_color_manual(values = custom_pal[1]) +
    scale_fill_manual(values = "white") +
    theme(axis.title=element_text(size=8),
          axis.title.x=element_text(margin = margin(t = 5, r = 0, b = 0, l = 0)),
          axis.title.y=element_text(margin = margin(t = 0, r = 0, b = 0, l = 0)),
          legend.title=element_text(size=8),
          legend.text=element_text(size=8),
          legend.position=c(0.20, 0.70),
          legend.margin=margin(c(0,0,0,0)),
          axis.text=element_text(size=8),
          plot.caption = element_text(hjust=0.5, size=8, face="italic")
         )

In [ ]:
ggsave("export/plot_average_900.pdf", 
       plot = last_plot(), 
       width = 40,
       height = 45,
       units = "mm",
       family = "Linux Libertine Display")

In [ ]:
options(repr.plot.width=2, repr.plot.height=3)
df_res %>%
    filter(aggregation_type == "zero-order hold", aggregation_interval == 900) %>%
    ggplot(aes(x=factor(max_peak), y=R, fill='constant', color='constant')) +
    ylim(-18, 18) +
    guides(fill=FALSE, color=FALSE) +
    geom_boxplot(outlier.shape = NA, lwd=.4) +
    geom_point(alpha=0.4, position = position_jitterdodge(), size=.5) +
    labs(x="target maximum peak [kW]", y="R [kWh]", color="Max Peak", caption="zero-order hold") +
    custom_theme +
    scale_color_manual(values = custom_pal[2]) +
    scale_fill_manual(values = "white") +
    theme(axis.title=element_text(size=8),
          axis.title.x=element_text(margin = margin(t = 5, r = 0, b = 0, l = 0)),
          axis.title.y=element_text(margin = margin(t = 0, r = 0, b = 0, l = 0)),
          legend.title=element_text(size=8),
          legend.text=element_text(size=8),
          legend.position=c(0.20, 0.70),
          legend.margin=margin(c(0,0,0,0)),
          axis.text=element_text(size=8),
          plot.caption = element_text(hjust=0.5, size=8, face="italic")
         )

In [ ]:
ggsave("export/plot_zero_order_hold_900.pdf", 
       plot = last_plot(), 
       width = 40,
       height = 45,
       units = "mm",
       family = "Linux Libertine Display")

### Details on Specific Scenario

Calculated C values for `max_peak = 40`, `aggregation_type = average` and `aggregation_interval = 900`

In [ ]:
options(repr.plot.width=6, repr.plot.height=3)
df_res %>%
    filter(aggregation_type == "average", max_peak == 40, aggregation_interval == 900) %>%
    ggplot(aes(x=day, y = C)) +
    geom_line() +
    geom_point(alpha=0.5, color= custom_pal[1]) +
    custom_theme

Distribution of calculated battery capacities.

In [ ]:
options(repr.plot.width=4, repr.plot.height=3)

plot_var <- df_res %>% 
    filter(aggregation_type == "average", max_peak == 40, aggregation_interval == 900)

qs = quantile(plot_var$C, probs = c(0.5, 0.9, 0.99))

ggplot(plot_var, aes(x=C)) +
    geom_density() +
    geom_vline(xintercept=qs, linetype = "longdash", color = custom_pal[1]) +
    annotate(geom="text", x=qs+5, y=0.08, label=names(qs)) +
    custom_theme

# Table Output

In [ ]:
df_res %>% 
    group_by(aggregation_type, max_peak, aggregation_interval) %>%
    summarize(max_C = max(C)) %>%
    left_join(df_res %>%
        filter(aggregation_interval == 5) %>%
        group_by(aggregation_type, max_peak) %>%
        summarize(C_true = max(C)) %>%
        select(aggregation_type, max_peak, C_true), by = c("aggregation_type", "max_peak")) %>%
    mutate(R = round(max_C - C_true, 2)) %>%
    filter(aggregation_interval %in% c(5, 300, 900, 1800, 3600), max_peak %in% c(38, 40)) %>%
    select(aggregation_type, max_peak, aggregation_interval, max_C, R)%>%
    gather(variable, value, max_C, R) %>%
    unite(tmp, aggregation_type, variable) %>%
    spread(tmp, value) -> table_var

In [ ]:
table_var = table_var %>%
    filter(aggregation_interval != 5) %>%
    select(max_peak, aggregation_interval, "zero-order hold_max_C", "zero-order hold_R", average_max_C, average_R);
table_var

In [ ]:
print(xtable(table_var),
      include.rownames=FALSE,
      include.colnames = FALSE,
      only.contents = TRUE, 
      booktabs = TRUE, 
      hline.after = 4,
      file = "export/table_C_comparison.tex")